## Pretrain a model that classifies fraudulent emails

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

kaggle_pt = torch.load('kaggle_hidden_states.pt').float()
labels_pt = torch.from_numpy(np.array(torch.load('kaggle_labels.pt'))).float()
print(f"kaggle_pt.shape: {kaggle_pt.shape}, labels_pt.shape: {labels_pt.shape}")

kaggle_pt.shape: torch.Size([11928, 64, 768]), labels_pt.shape: torch.Size([11928])


`kaggle_pt` represents the kaggle hidden states observed by the pre-trained BERT model

With the shape of [n, 64, 768]

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, dropout_rate=0.2):
        super(MLP, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, dilation=2)
        self.fc1 = nn.Linear(32 * 382, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.fc2 = nn.Linear(512, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(p=dropout_rate)
        self.fc3 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool1d(x, kernel_size=2)  # Add max pooling layer
        x = x.view(-1, 32 * 382)
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        output = self.fc3(x)
        output = self.sigmoid(output)
        return output.squeeze(dim=1)

In [24]:
cuda_device = "cuda:3"
# define your model
model = MLP().to(cuda_device)
# define your loss function
criterion = nn.BCELoss()
# define your optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [26]:
# kaggle_pt[:32].shape, labels_pt[:32].shape, model(kaggle_pt[:32]).shape
# loss = criterion(model(kaggle_pt[:32]), labels_pt[:32].float())
# loss.item()

In [28]:
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score

def train(model, inputs, labels, optimizer, criterion, epochs, batch_size):
    # split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(inputs, labels, test_size=0.1, random_state=42)

    num_train_samples = X_train.shape[0]
    num_val_samples = X_val.shape[0]

    num_train_batches = num_train_samples // batch_size
    num_val_batches = num_val_samples // batch_size

    for epoch in range(epochs):
        train_loss = 0.0
        val_loss = 0.0

        # loop over training batches
        for i in range(num_train_batches):
            start_idx = i * batch_size
            end_idx = (i + 1) * batch_size

            # get the current batch
            batch_inputs = X_train[start_idx:end_idx].to(cuda_device)
            batch_labels = y_train[start_idx:end_idx].to(cuda_device)

            optimizer.zero_grad()
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_labels)
            loss.backward(retain_graph=True)  # add retain_graph=True here
            optimizer.step()
            with torch.no_grad():
                train_loss += loss.item()

        # loop over validation batches
        for i in range(num_val_batches):
            start_idx = i * batch_size
            end_idx = (i + 1) * batch_size

            # get the current batch
            batch_inputs = X_val[start_idx:end_idx].to(cuda_device)
            batch_labels = y_val[start_idx:end_idx].to(cuda_device)

            # evaluate the model on validation set
            with torch.no_grad():
                outputs = model(batch_inputs)
                loss = criterion(outputs, batch_labels)
                val_loss += loss.item()

        train_loss /= num_train_batches
        val_loss /= num_val_batches

        if epoch % 1 == 0:
            # get the predictions on the training set and calculate f1 score
            with torch.no_grad():
                train_preds = model(X_train.to(cuda_device)).cpu().numpy()
                train_f1 = f1_score(y_train.cpu().numpy(), (train_preds >= 0.5).astype(int))

            # get the predictions on the validation set and calculate f1 score, auc roc, and auc pr
            with torch.no_grad():
                val_preds = model(X_val.to(cuda_device)).cpu().numpy()
                val_f1 = f1_score(y_val.cpu().numpy(), (val_preds >= 0.5).astype(int))
                val_auc_roc = roc_auc_score(y_val.cpu().numpy(), val_preds)
                val_auc_pr = average_precision_score(y_val.cpu().numpy(), val_preds)
                    
            print(f"Epoch: {epoch+1}, Train BCE Loss: {train_loss}, Val BCE Loss: {val_loss}, F1 Score: {train_f1}/{val_f1}, AUC ROC: {val_auc_roc}, AUC PR: {val_auc_pr}")



# train your model for 100 epochs
train(model, kaggle_pt, labels_pt, optimizer, criterion, epochs=100, batch_size=256)

Epoch: 1, Train Loss: 0.2751535820524867, Val Loss: 0.21252139285206795
Epoch: 2, Train Loss: 0.19493088962101354, Val Loss: 0.18338479846715927
Epoch: 3, Train Loss: 0.16698520139950077, Val Loss: 0.16232554614543915
Epoch: 4, Train Loss: 0.14697592869037535, Val Loss: 0.14718198776245117
Epoch: 5, Train Loss: 0.13176632908786215, Val Loss: 0.13335424289107323
Epoch: 6, Train Loss: 0.11886289425012542, Val Loss: 0.12301931902766228
Epoch: 7, Train Loss: 0.10734970940322411, Val Loss: 0.11503180488944054
Epoch: 8, Train Loss: 0.09875089947770281, Val Loss: 0.10624210350215435
Epoch: 9, Train Loss: 0.0901965509827544, Val Loss: 0.1005475539714098


KeyboardInterrupt: 

In [10]:
kaggle_pt.shape, labels_pt.shape

(torch.Size([11928, 64, 768]), (11928,))

In [42]:
labels_pt[:-1].shape

torch.Size([11928])